# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

print("\n################################################################################\n")

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Sat Dec  3 00:05:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## GitHub

In [ ]:
!ls
!git clone https://github.com/YunZhi246/subpopulation-data-poisoning-attacks.git

drive  sample_data
Cloning into 'subpopulation-data-poisoning-attacks'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 137 (delta 77), reused 98 (delta 41), pack-reused 0
Receiving objects: 100% (137/137), 1.61 MiB | 6.36 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [ ]:
%cd /content/subpopulation-data-poisoning-attacks

/content/subpopulation-data-poisoning-attacks


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 3.9 MB/s 
     |████████████████████████████████| 7.0 MB 18.2 MB/s 
     |████████████████████████████████| 26.0 MB 155.6 MB/s 
     |████████████████████████████████| 215 kB 111.5 MB/s 
     |████████████████████████████████| 4.4 MB 27.4 MB/s 
     |████████████████████████████████| 4.3 MB 86.3 MB/s 
     |████████████████████████████████| 451 kB 102.6 MB/s 
     |████████████████████████████████| 6.6 MB 35.2 MB/s 
     |████████████████████████████████| 182 kB 106.3 MB/s 
     |████████████████████████████████| 132 kB 109.3 MB/s 
     |████████████████████████████████| 212 kB 112.3 MB/s 
     |████████████████████████████████| 127 kB 97.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: scipy
    Found 

## Notebook

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

In [ ]:
# os.chdir('../../')
# os.environ['ML_DATA'] = ''

In [ ]:
import torch
import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import TensorDataset, DataLoader

from attack_nlp import init_cluster_attack
from subclass_avail import common
from subclass_avail.target_nlp import bert_utils

In [ ]:
# Constants

seed = 42
batch = 4
lr = 1e-5
epochs = 4

# Run specific parameters
pois_rate = 2.0
cl_ind = 29
n_remove = 150
base_def = 0.987879

m_type = 'FT'
frozen = False

In [ ]:
device = bert_utils.get_device()
bert_utils.set_seed(device, seed=seed)  # Seed all the PRNGs

Available device:  cuda


# Clustering

In [ ]:
common.create_dirs()

model_name = bert_utils.get_model_name()
victim_dir = "/content/drive/MyDrive/storage/other/saved_models/victims_2.0"
save_path = os.path.join(
    victim_dir,
    'attack_setup_{}_{}.npy'.format(model_name, pois_rate)
)
setup_params = np.load(save_path, allow_pickle=True).item()

all_inds = setup_params["all_inds"]
labels = setup_params["labels"]
labels_t = setup_params["labels_t"]
labels_ho = setup_params["labels_ho"]
preds_ho = setup_params["preds_ho"]
x = setup_params["x"]
x_att = setup_params["x_att"]
x_ho = setup_params["x_ho"]
x_ho_att = setup_params["x_ho_att"]
x_t = setup_params["x_t"]
x_t_att = setup_params["x_t_att"]
y = setup_params["y"]
y_t = setup_params["y_t"]
y_ho = setup_params["y_ho"]

print('\nx shape: {}\nx_ho shape:{}\nx_t shape: {}'.format(x.shape, x_ho.shape, x_t.shape))


x shape: (12500, 256)
x_ho shape:(12500, 256)
x_t shape: (25000, 256)


In [ ]:
trn_inds = np.where(labels == cl_ind)[0]
tst_inds = np.where(labels_t == cl_ind)[0]
ho_inds = np.where(labels_ho == cl_ind)[0]
pois_inds = np.random.choice(
    ho_inds,
    int(ho_inds.shape[0] * pois_rate),
    replace=True
)
print("cluster ind:", cl_ind)
print("train cluster size:", trn_inds.shape[0])
print("test cluster size:", tst_inds.shape[0])
print("pois cluster size", pois_inds.shape[0])
trn_x = x
trn_y = y
trn_x_att = x_att

preds_cl = preds_ho[ho_inds].sum(axis=0)
assert preds_cl.size == 2

worst_class = np.argmin(preds_cl)
print(worst_class, preds_cl)

pois_x = np.take(x_ho, pois_inds, axis=0)
pois_y = np.take(y_ho, pois_inds, axis=0)
pois_x_att = np.take(x_ho_att, pois_inds, axis=0)

pois_y[:] = worst_class  # Assigns the worst class label to every poison point
trn_x = np.concatenate((trn_x, pois_x))
trn_y = np.concatenate((trn_y, pois_y))
trn_x_att = np.concatenate((trn_x_att, pois_x_att))
rand_inds = np.random.choice(trn_x.shape[0], trn_x.shape[0], replace=False)
xt_p, xt_p_att, yt_p = x_t[tst_inds], x_t_att[tst_inds], y_t[tst_inds]

# Create the subset of the test set not containing the targeted
# sub population to compute the collateral damage
x_coll = x_t[[i for i in range(x_t.shape[0]) if i not in tst_inds]]
x_coll_att = x_t_att[[i for i in range(x_t_att.shape[0]) if i not in tst_inds]]
y_coll = y_t[[i for i in range(y_t.shape[0]) if i not in tst_inds]]
print('\nx coll shape: {}\nx_att coll shape:{}\ny coll shape: {}'.format(
    x_coll.shape, x_coll_att.shape, y_coll.shape))


cluster ind: 29
train cluster size: 64
test cluster size: 165
pois cluster size 174
1 [ 317.90230727 -317.40503192]

x coll shape: (24835, 256)
x_att coll shape:(24835, 256)
y coll shape: (24835,)


# Trim

In [ ]:
def trim(dataset, size, x, x_att, y, num_remove):
    inds = []
    new_inds = list(range(x.shape[0]))
    it = 0
    while sorted(new_inds) != sorted(inds) and it < 5:
        print('Trim iteration: {}\n'.format(it))
        it += 1
        inds = new_inds[:]
        
        # model = train_model(dataset, model_type, trn_x, trn_y, tst_x, tst_y)  
        model = bert_utils.wrap_train(
            x,
            y,
            x_att,
            b_size=batch,
            lr=lr,
            epochs=epochs,
            frozen=frozen
        ) 
        
        # Prepare torch loader
        device = bert_utils.get_device()

        test_ds = TensorDataset(
            torch.from_numpy(x),
            torch.from_numpy(x_att),
            torch.from_numpy(y)
        )
        test_dl = DataLoader(test_ds, shuffle=False, batch_size=batch)
         
        # preds = model.predict(x)
        _, preds_raw = bert_utils.predict_bert(model, device, test_dl, raw=True)
        preds = [np.array(p) for p in preds_raw]
        preds = np.concatenate(preds)
        
        # One hot encode labels
        onehoty = OneHotEncoder().fit_transform(y.reshape(-1, 1)).toarray()
        
        probs = np.multiply(preds, onehoty).sum(axis=1)
        new_inds = np.argpartition(probs, num_remove)[num_remove:]
    return model, new_inds

In [ ]:
%%time
trim_model, trim_inds = trim(
    dataset='imdb',
    size=256,
    x=trn_x[rand_inds],
    x_att=trn_x_att[rand_inds],
    y=trn_y[rand_inds],
    num_remove=n_remove
)

Trim iteration: 0

Available device:  cuda


Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Epoch 0 of 4


...

Train loss at epoch 2: 0.08410414163297356
Training accuracy - epoch 2: 0.9930577469233196
Epoch 3 of 4


100%|██████████| 3169/3169 [15:16<00:00,  3.46it/s]


Train loss at epoch 3: 0.041680796308409596
Training accuracy - epoch 3: 0.9962133165036289
Saving to bert_tuned
Available device:  cuda


100%|██████████| 3169/3169 [05:39<00:00,  9.34it/s]

CPU times: user 8h 7min 43s, sys: 1h 21min 2s, total: 9h 28min 45s
Wall time: 7h 26min 22s


In [ ]:
# Save trim model
save_pth = os.path.join(
    victim_dir,
    'imdb_bert_{}_TRM_pois{}_ind{}.ckpt'.format(m_type, pois_rate, cl_ind)
)
print(save_pth)

torch.save(trim_model.state_dict(), save_pth)

/content/drive/MyDrive/storage/other/saved_models/victims_2.0/imdb_bert_FT_TRM_pois2.0_ind29.ckpt


In [ ]:
# Evaluation

pois_ds = TensorDataset(
    torch.from_numpy(xt_p),
    torch.from_numpy(xt_p_att),
    torch.from_numpy(yt_p)
)

pois_dl = DataLoader(pois_ds, shuffle=False, batch_size=batch)

In [ ]:
trim_acc = bert_utils.predict_bert(trim_model, device, pois_dl, acc=True)

100%|██████████| 42/42 [00:04<00:00,  9.56it/s]


In [ ]:
print('Accuracy of Trim model on the poisoned data: {}'.format(trim_acc))

Accuracy of Trim model on the poisoned data: 0.9030303030303031


In [ ]:
print('New targeted damage: {}'.format(base_def - trim_acc))

New targeted damage: 0.08484869696969688
